In [ ]:
import cv2
from IPython.display import clear_output


In [ ]:
cap = cv2.VideoCapture('../data/tienda.mp4')


In [ ]:
videopath = '../../data/artifacts/videos/tienda.mp4'

%pylab inline 

import imutils
vid = cv2.VideoCapture(videopath)

print_frame = 500
while(True):
    ret, frame = vid.read()
#for ii in range(10):
    if print_frame > 0:
        print_frame -= 1
        continue
    else:
        print_frame = 500
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        def rotate_image(image, angle):
          image_center = tuple(np.array(image.shape[1::-1]) / 2)
          rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
          result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
          return result
        
        def round_up(n, decimals=0):
            multiplier = 10 ** decimals
            return math.ceil(n * multiplier) / multiplier


        image = frame
        image = rotate_image(image, -28)
        image = image[450:650, 680:1650]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,3)

        mask = thresh.copy()
        mask = cv2.merge([mask,mask,mask])

        picture_threshold = image.shape[0] * image.shape[1] * .001
        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < picture_threshold:
                cv2.drawContours(mask, [c], -1, (0,0,0), -1)

        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        result = cv2.bitwise_xor(thresh, mask)

        text_pixels = cv2.countNonZero(result)
        percentage = (text_pixels / (image.shape[0] * image.shape[1])) * 100

        fig=figure(figsize=(24, 16))
        title('Porcentaje de vitrina completa: {:.2f}%'.format(100-round_up(percentage,-1)))
        columns = 1
        rows = 2
        fig.add_subplot(rows, columns, 1)
        plt.imshow(thresh)
        fig.add_subplot(rows, columns, 2)
        plt.imshow(frame)
        

        def get_axis_limits(ax, scale=.9):
            return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale

        plt.show()
        #imshow(thresh)
        #imshow(frame)

        #show()
        clear_output(wait=True)